In [3]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator

In [4]:
# Binary function:
def to_binary(number, nbits = None):

    output = bin(number)[2:]
    if nbits == None:
        return output # Array indexing to delete "0b"
    elif nbits > len(output):
        while nbits > len(output):
            output = '0' + output
        return output
    else:
        return output



In [5]:
# Multi-Control-Z function
def multi_control_z(n_qubits):

    qc = QuantumCircuit(n_qubits)
    qc.h(n_qubits - 1)
    qc.mcx(list(range(n_qubits - 1)), n_qubits - 1) # 1st parameter: list of controlling qubits | 2nd parameter: target qubit
    qc.h(n_qubits - 1)

    return qc


In [6]:
# Diffuser
def diffuser_circuit(n_qubits):

    qc = QuantumCircuit(n_qubits)
    for qb in range (n_qubits):
        qc.h(qb)
    for qb in range (n_qubits):
        qc.x(qb)
    multi_z = multi_control_z(n_qubits)
    qc.append(multi_z.to_gate(),  range(n_qubits-1, -1, -1))  
    for qb in range (n_qubits):
        qc.x(qb)
    for qb in range (n_qubits):
        qc.h(qb)

    return qc


In [7]:
# Single qubit pi phase adder function
def pi_phase_adder():

    qc = QuantumCircuit(1)
    qc.z(0)
    qc.x(0)
    qc.z(0)
    qc.x(0) 

    return qc


In [8]:
def less_than_oracle(number, n_qubits):

    qc = QuantumCircuit(n_qubits)
    number_binary = to_binary(number, n_qubits)
    number_binary = number_binary.rstrip('0')

    # Circuit to create "less than" oracle

    # If the most significant qubit is 1, apply Z (not controlled)
    if number_binary[0] == '1':
        qc.x(n_qubits - 1)
        qc.z(n_qubits - 1)
        qc.x(n_qubits - 1)
    else:
        qc.x(n_qubits - 1)

    # For loop on the remaining digits
    for position1, value in enumerate(number_binary[1:]):
        # Rename the position as it starts with 0 in the second bit and
        # we want it to be 1.
        position = position1 + 1
        #for nq in range(nqubits):
        #    circuit.barrier(nq)

        if value == '0':
            # If the digit is 0
            # Just apply a X gate
            qc.x(n_qubits-position-1)
        else:
            # If the digit bi is 1
            # Apply a multi-controlled Z gate to mark states of the shape:
            # |bn...bi+1 0 qi-1...q1>
            # where bn,...,bi+1 are the first n-i bits of m, which is of the shape bn...bi+1 1 bi-1...b1
            # because we just checked that bi is 1.
            # Hence, the numbers of the form bn...bi+1 0 qi-1...q1 are smaller than m.
            qc.x(n_qubits-position-1)
            multi_z = multi_control_z(position + 1)
            qc.append(multi_z.to_gate(), range(n_qubits-1, n_qubits-position-2, -1))
            qc.x(n_qubits-position-1)

    for position, value in enumerate(number_binary):
        # Apply X gates to qubits in position of bits with a 0 value
        #for nq in range(nqubits):
        #    circuit.barrier(nq)
        if value == '0':
            qc.x(n_qubits-position-1)
        else:
            pass
    
    return qc

In [9]:
# Greater than oracle function
def greater_than_oracle(number, n_qubits):

    qc = QuantumCircuit(n_qubits)

    # Create less than and global pi phase adder circuits
    less_than = less_than_oracle(number=number, n_qubits=n_qubits)
    pi_phase = pi_phase_adder()

    # Append them to the general circuit
    qc.append(less_than.to_gate(),  range(0,n_qubits, 1))
    qc.append(pi_phase.to_gate(), range(0, -1, -1)) # This range is only outputting 0, so the pi phase is only added to the LSQubit

    return qc


In [10]:
# Range fuction
def range_of_oracle(lower_number, higher_number, n_qubits):

    qc = QuantumCircuit(n_qubits)

    # Create greater and less than oracles based on the numbers input
    less_than = less_than_oracle(higher_number, n_qubits)
    greater_than = greater_than_oracle(lower_number, n_qubits)
    pi_phase = pi_phase_adder()

    # Append them to the general circuit
    qc.append(greater_than.to_gate(), range(0, n_qubits, 1))
    qc.append(less_than.to_gate(), range(0, n_qubits, 1))
    qc.append(pi_phase.to_gate(), range(0, -1, -1))

    return qc


In [11]:
# Range program: Creating a circuit where the range oracle is implemented and simulated
def range_of_program(lower_number, higher_number, n_qubits):

    cr = ClassicalRegister(n_qubits)
    qr = QuantumRegister(n_qubits)
    qc = QuantumCircuit(qr, cr)

    qc.h(qr)
    range_of = range_of_oracle(lower_number=lower_number, higher_number=higher_number, n_qubits=n_qubits)
    diffuser = diffuser_circuit(n_qubits)

    qc.append(range_of.to_gate(), range(0, n_qubits, 1))
    qc.append(diffuser.to_gate(), range(0, n_qubits, 1))
    qc.measure(qr,cr)

    '''So the output only shows the numbers within the range even if the other numbers get some shots)
    sq = 2**n_qubits
    fr=sq/(higher_number-lower_number)
    numit=round(fr)+1
    if numit == 0:
        numit=1
    #numit=6  
    print (numit)
    for i in range(numit):
        qc.append(range_of.to_gate(), range(0, n_qubits, 1))
        qc.append(diffuser.to_gate(), range(0, n_qubits, 1))
    qc.measure(qr,cr)'''

    return qc


In [26]:
def program_less_than (nqubits,number_less_than):
  # This program creates the circuit of a qiskit program that produces a quantum states of nqubits qbits
  # where all the values of the state less than number_less_than have the greater amplitude
  qreg = QuantumRegister(nqubits)
  creg = ClassicalRegister(nqubits)
  qprogram = QuantumCircuit(qreg,creg)
  qprogram.h(qreg)
  diffuser = diffuser_circuit(nqubits)
  less_than = less_than_oracle(number_less_than, nqubits)

  qprogram.append(less_than.to_gate(), range(0, nqubits, 1))
  qprogram.append(diffuser.to_gate(), range(0, nqubits, 1))
  qprogram.measure(qreg,creg)

  '''
  sq = 2**n_qubits
  fr=sq/(higher_number-lower_number)
  numit=round(fr)+1
  if numit == 0:
      numit=1
  #numit=6  
  print (numit)
  for i in range(numit):
      qc.append(range_of.to_gate(), range(0, n_qubits, 1))
      qc.append(diffuser.to_gate(), range(0, n_qubits, 1))
  qc.measure(qr,cr)'''
  
  return qprogram


nqubits = 4
number_less_than = 4

qp = program_less_than(nqubits, number_less_than)
qp.draw()

method = "statevector"
sim = AerSimulator(method = method)

transpiled_circuit = transpile(qp, backend = sim)

nshots = 200
sim_run = sim.run(transpiled_circuit, shots = nshots)
sim_result=sim_run.result()
counts_result = sim_result.get_counts(qp)

# Display histogram as ASCII in terminal
max_count = max(counts_result.values())
for state, count in sorted(counts_result.items(), key=lambda x: int(x[0], 2)):
    bar = '█' * int((count / max_count) * 50)
    print(f"{state}: {bar} ({count})")

print('''\nPrinting the various results followed by how many times they happened (out of the {} cases):\n'''.format(nshots), flush = True)
for i in range(len(counts_result)):
  print('-> Result \"{0}\" happened {1} times out of {2}'.format(
  list(sim_result.get_counts().keys())[i],
  list(sim_result.get_counts().values())[i],nshots), 
  flush = True)

qp.draw()

0000: ██████████████████████████████████████████████████ (58)
0001: ████████████████████████████████████████ (47)
0010: ██████████████████████████████████████████████ (54)
0011: ███████████████████████████████████ (41)

Printing the various results followed by how many times they happened (out of the 200 cases):

-> Result "0011" happened 41 times out of 200


-> Result "0001" happened 47 times out of 200
-> Result "0000" happened 58 times out of 200
-> Result "0010" happened 54 times out of 200


┌───┐┌──────────────┐┌──────────────┐┌─┐         
q25_0: ┤ H ├┤0             ├┤0             ├┤M├─────────
       ├───┤│              ││              │└╥┘┌─┐      
q25_1: ┤ H ├┤1             ├┤1             ├─╫─┤M├──────
       ├───┤│  circuit-927 ││  circuit-923 │ ║ └╥┘┌─┐   
q25_2: ┤ H ├┤2             ├┤2             ├─╫──╫─┤M├───
       ├───┤│              ││              │ ║  ║ └╥┘┌─┐
q25_3: ┤ H ├┤3             ├┤3             ├─╫──╫──╫─┤M├
       └───┘└──────────────┘└──────────────┘ ║  ║  ║ └╥┘
c24: 4/══════════════════════════════════════╩══╩══╩══╩═
                                             0  1  2  3